In [ ]:
import os
import torch
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from torchvision.transforms import functional as F
import os
import csv
import sys
import numpy as np
import cv2          
import time
from torchvision import transforms
import torch
import torch.optim as optim
from torchvision.models.detection import fasterrcnn_resnet50_fpn, FasterRCNN_ResNet50_FPN_Weights
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from transformers import DeformableDetrForObjectDetection, AutoImageProcessor
from transformers import DetrImageProcessor, DetrForObjectDetection, DetrConfig
from torch.utils.data import Dataset
import torchvision
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from torchvision.models.detection import fasterrcnn_resnet50_fpn
import torchvision.transforms as T
!pip install transformers --upgrade
!pip install pycocotools
from torch.utils.data import DataLoader
from PIL import Image

# Define the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
processor = DetrImageProcessor.from_pretrained("facebook/detr-resnet-50")

def load_model(checkpoint_path):
    model = DeformableDetrForObjectDetection.from_pretrained("SenseTime/deformable-detr")
    checkpoint = torch.load(checkpoint_path)
    model.load_state_dict(checkpoint['model_state_dict'])
    return model

def visualize_predictions(image, logits, pred_boxes, threshold=0.8):
    plt.figure(figsize=(10, 10))
    plt.imshow(image)
    ax = plt.gca()
#     print(type(image))
    width, height = image.size
    logits = logits.cpu()
    pred_boxes = pred_boxes.cpu()
#     print(image.type)

#     scores = logits.softmax(-1).max(-1).values
#     print(logits)

    for box, score  in zip(pred_boxes, logits):
        if score.item() > threshold:
            x1, y1, w, h = box
            x1=x1*width
            w=w*width
            y1=y1*height
            h=h*height
            x1=x1-w/2
            y1=y1-h/2
            x2, y2 = x1 + w, y1 + h
            rect = patches.Rectangle((x1, y1), x2-x1, y2-y1, linewidth=1, edgecolor='r', facecolor='none')
            ax.add_patch(rect)
#             score_str = "{:.2f}".format(score.item())
#             plt.text(x1, y1, score_str, color='white', fontsize=12, bbox=dict(facecolor='red', alpha=0.5))

    plt.axis('off')
    plt.show()

def main():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    checkpoint_path = "/kaggle/input/deformable-detr/model.pt"
    model = load_model(checkpoint_path)
    model.to(device)
    model.eval()
    processor = DetrImageProcessor.from_pretrained("facebook/detr-resnet-50")

    # Define the image folder and get a list of images
    img_folder='/kaggle/input/cv-a4-coco-dataset/coco_1k/val2017'
    image_files = [os.path.join(img_folder, img) for img in os.listdir(img_folder) if img.endswith(('.png', '.jpg', '.jpeg'))]

    # Process images one by one
    i=0
    for image_path in image_files:
        i+=1
        image = Image.open(image_path)
        inputs = processor(images=image, return_tensors="pt").to(device)
        
        with torch.no_grad():
            outputs = model(inputs['pixel_values'])
        logits = outputs['logits']
        pred_boxes = outputs['pred_boxes']
        logits = torch.nn.functional.softmax(logits, dim=2)
        print(logits.shape)
        print(pred_boxes.shape)
        tensor_np = logits.cpu().numpy()

        # Find the indices of maximum values along axis 2 (last axis)
        max_indices = tensor_np.argmax(axis=2)

        # Check if the maximum value is at the 0th position
        keep_indices = (max_indices == 0)

        # Convert the keep indices back to a PyTorch tensor
        keep_indices_tensor = torch.tensor(keep_indices, dtype=torch.bool)

        # Filter the tensor and pred_boxes using the keep indices
        filtered_logits = logits[keep_indices_tensor]
        filtered_pred_boxes = pred_boxes[keep_indices_tensor.unsqueeze(2).expand_as(pred_boxes)].view(-1, 4)

        # Print the filtered tensor and pred_boxes
#         print("Filtered tensor:", filtered_logits)
        print(filtered_logits.shape)
        filtered_logits = filtered_logits[:, 0].unsqueeze(1)
#         print("Filtered tensor:", filtered_logits)
#         print("Filtered pred_boxes:", filtered_pred_boxes)
        print(filtered_logits.shape)
        print(filtered_pred_boxes.shape)
        # Convert outputs to CPU for visualization if on GPU
#         outputs = {k: v.to('cpu') for k, v in outputs.items()}
#         print(outputs)
        visualize_predictions(image, filtered_logits, filtered_pred_boxes)
#         if(i>3):
#             break
        break

if __name__ == "__main__":
    main()
